In [207]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [208]:
#Load CSV from part 1 to dataframe
weather_df = pd.read_csv('output_data/cities.csv')


In [252]:
#Humidity Heat Map
fig = gmaps.figure(layout={'width': '600x', 'height': '500px'})

locations = weather_df[['Lat','Lng']]
humidity = weather_df['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='500px', width='600x'))

In [253]:
ideal_weather_df = weather_df.loc[
    (weather_df['Max Temp']<=85) & 
    (weather_df['Max Temp']>=70) & 
    (weather_df['Wind Speed']<=6) & 
    (weather_df['Humidity']<=65) &
    (weather_df['Cloudiness']>=25)
]
     
print(len(ideal_weather_df))
ideal_weather_df

4


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,huanren,41.26,125.37,83.75,56,77,2.17,CN,2020-08-29 21:36
55,nazas,25.23,-104.13,81.23,42,100,4.05,MX,2020-08-29 21:36
110,airai,-8.93,125.41,74.93,43,86,4.92,TL,2020-08-29 21:37
165,riyadh,24.69,46.72,80.60,21,92,5.82,SA,2020-08-29 21:37


In [254]:
# locations = []
hotels = []

# for ea in (np.arange(0,len(ideal_weather_df))):
#     locations.append(f"{ideal_weather_df.iloc[ea,1]}, {ideal_weather_df.iloc[ea,2]}")

for ea in (np.arange(0,len(ideal_weather_df))):
    city = ideal_weather_df.iloc[ea,0]
    country = ideal_weather_df.iloc[ea,7]
    location = f"{ideal_weather_df.iloc[ea,1]}, {ideal_weather_df.iloc[ea,2]}"
    target_type = 'Hotel'
    target_keyword = 'Hotel'
    target_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&keyword={target_keyword}&type={target_type}&radius=5000&key={g_key}"
    try:
        response = requests.get(target_url).json()
        hotel_name = response['results'][0]['name']
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']    
        hotels.append({'city': city, 'country': country, 'hotel_name': hotel_name, 'hotel_lat': lat, 'hotel_lng': lng})
        
    except:
        print(f"No hotels listed within a 5000m radius of {city} {country}")
        
hotels_df = pd.DataFrame(hotels)
coords = hotels_df[['hotel_lat','hotel_lng']]
names = []
for ea in (np.arange(0,len(hotels_df))):
    names.append("Hotel: "+hotels_df.iloc[ea,2]+", City: " +hotels_df.iloc[ea,0]+", Country: "+hotels_df.iloc[ea,1])
    
markers = gmaps.marker_layer(coords,info_box_content=names, display_info_box=True)
fig.add_layer(markers)

fig

No hotels listed within a 5000m radius of airai TL


Figure(layout=FigureLayout(height='500px', width='600x'))